In [1]:
from textblob.classifiers import NaiveBayesClassifier
import pandas
import re
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 
from textblob import TextBlob
from textblob import classifiers
from textblob.sentiments import NaiveBayesAnalyzer
from nltk.corpus import stopwords #import stopwords
from nltk.tokenize import RegexpTokenizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer
import re #import regular expression
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from collections import Counter

In [150]:
df = pandas.read_csv('weather-agg-DFE.csv', usecols =["tweet_text","what_emotion_does_the_author_express_specifically_about_the_weather"])
test = pandas.read_csv('weather-evaluated-agg-DFE.csv', usecols =["tweet_text","sentiment"])
text = df["tweet_text"].tolist()
sentiment = df["what_emotion_does_the_author_express_specifically_about_the_weather"].tolist()
text_test = test["tweet_text"].tolist()
sentiment_test = test["sentiment"].tolist()

df.head()

print(len(text))

1000


In [156]:
def preprocessing(data):        
        sentence = ' '.join(re.sub('@[^\s]+','',data).split())   #removeMention
        sentence = ' '.join(re.sub(r"http\S+", '', sentence).split())  #removeLink
        sentence = ' '.join(re.sub('#\S+', '', sentence).split())  #removeHashtag
        sentence = ' '.join(re.sub('RT|cc', '', sentence).split())  #removeRT/cc
        sentence = ' '.join(re.sub('[^a-zA-Z]', " ", sentence).split()) #removeSymbol
        sentence = ' '.join(re.sub('(\\b[A-Za-z] \\b|\\b [A-Za-z]\\b)', '', sentence).split()) #removeOneChar
        sentence = ' '.join(re.sub("\s\s+" , " ", sentence).split()) #removeMultiSpace
        sentence = sentence.lower().split()
        return sentence
    
cleaned= []
i = 0
while i < len(text):
    result = preprocessing(text[i])
    result = ' '.join(result)
    cleaned.append(result)
    i += 1
    
print(cleaned)
print(len(cleaned))

['grilling kabobs on the grill last night was amazing so glad this weather is finally here', 'the slowest day ever and the weather makes it even worse', 'fire weather watch issued may at pm cdt expiring may at pm cdt by nws lubbock link', 'im going to lunch early today the weather is nice outside so im going to enjoy somewhere pleasant', 'weekend weather causes delays in bridge work link', 'passing out now working tonight storms today would be nice would be great sleeping weather mmmm', 'us gas warm weather forecasts lift natural gas futures link', 'friday at at tha stadium if tha weather bad its gone be in tha gym at', 'hate this weather good day for movie marathon though xmen', 'tgif biatches here to sunny friday regardless of rain and drama', 'good crowd in wrigley today terrible weather tell that redneck that just called in check the reds crowd on wed day game', 'the storm is only that which authenticates your path', 'in the house washing clothes on this rainy day', 'pm feels humid

In [157]:
def word_tokenization(s):
    tokens = word_tokenize(s)

    return tokens

stop_words = set(stopwords.words('english')) 
    
def stopword(str):
    word_tokens = word_tokenize(str) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    filtered_sentence = [] 
    
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

sw = []
i = 0
while i < len(text):
    baru = ' '.join(stopword(cleaned[i]))
    sw.append(baru)
    i += 1


In [153]:
train = []
j = 0
while j < len(df):
    data_train = (sw[j], sentiment[j])
    train.append(data_train)
    j += 1

print(train)

[('grilling kabobs grill last night amazing glad weather finally', 'Positive'), ('slowest day ever weather makes even worse', 'Negative'), ('fire weather watch issued may pm cdt expiring may pm cdt nws lubbock link', 'Neutral / author is just sharing information'), ('im going lunch early today weather nice outside im going enjoy somewhere pleasant', 'Positive'), ('weekend weather causes delays bridge work link', 'Neutral / author is just sharing information'), ('passing working tonight storms today would nice would great sleeping weather mmmm', 'Positive'), ('us gas warm weather forecasts lift natural gas futures link', 'Neutral / author is just sharing information'), ('friday tha stadium tha weather bad gone tha gym', 'Negative'), ('hate weather good day movie marathon though xmen', 'Negative'), ('tgif biatches sunny friday regardless rain drama', 'Positive'), ('good crowd wrigley today terrible weather tell redneck called check reds crowd wed day game', 'Negative'), ('storm authentic

In [154]:
cl = NaiveBayesClassifier(train)

In [155]:
from textblob import TextBlob
blob = TextBlob("My husband love fishing. "
                "But the weather was bad. He was not happy.",
                classifier=cl)
for sentence in blob.sentences:
    print(str(sentence)+" : "+str(sentence.classify()))

My husband love fishing. : Positive
But the weather was bad. : Negative
He was not happy. : Negative


In [162]:
countacc = 0

clntwttes = []
i = 0
while i < len(test):
    baru = preprocessing(text_test[i])
    baru = ' '.join(baru)
    clntwttes.append(baru)
    i += 1
    
    
tee = []
i = 0
while i < len(test):
    baru = ' '.join(stopword(clntwttes[i]))
    tee.append(baru)
    i += 1

mx = NaiveBayesClassifier(train)
j = 0
while j < len(test):
    blob = TextBlob(tee[j], classifier=cl)
    print(blob.classify() + "---" + sentiment_test[j])
    if(blob.classify() == sentiment_test[j]) :
        countacc = countacc + 1
    j += 1

Positive---Positive
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Negative---Negative
Positive---Positive
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Positive
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing informa

Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Negative---Negative
Positive---Positive
Neutral / author is just sharing information---Neutral / author is jus

Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Negative---Negative
Negative---Neutral / author is just sharing information
Negative---Negative
Negative---Negative
Positive---Neutral / author is just sharing information
Negative---Negat

Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Negative---Negative
Negative---Negative
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Positive---Positive
Positive---Positive
Positive---Positive
Positive---Positive
Tweet not related to weather condition---Tweet not related to weather condition
Tweet not related to weather condition---Tweet not related to weather condition
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Tweet not related to weather condition
Positive---Positive
Negative---Negative
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Po

Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Negative---Negative
Negative---Negative
Negative---Negative
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Negative---Neutral / author is just sharing information
Tweet not related to weather condition---Tweet not related to weathe

Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Positive---Positive
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Positive---Positive
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Positive---Positive
Tweet not related to weather condition---Tweet not related to weathe

Positive---Positive
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Negative---Negative
Positive---Positive
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Positive---Positive
Positive---Positive
Negative---Negative
Positive---Positive
Positive---Neutral / author is just sharing information
Positive---Positive
Positive---Positive
Positive---Positive
Negative---Negative
Tweet not related to weather condition---Tweet not related to weather condition
Negative---Negative
Neutral / author is just sharing information---Neutral / author is just sharing information
Neutral / author is just sharing information---Neutral / author is just sharing information
Positive---Positive
Positive---Positive
Negative---Negative
Negative---Positive
Negative---Negative
Negative---Neutral / author 

930
